In [ ]:
#In this notebook we learned to play around with the Carto python API: cartoframes.
#Nothing too exciting here, though we did use the results of the last notebook cell to generate some cool
#cartodb animated maps.

In [1]:
from carto.auth import APIKeyAuthClient
from carto.exceptions import CartoException
from carto.sql import SQLClient
import cartoframes
import pandas as pd
import geopandas as gpd
from cartoframes import Layer, BaseMap, styling, QueryLayer
import datetime


In [2]:
# Neighborhood Tabulation Area defined neighborhoods (Census tract Stuff)
fname = "nycgiszoningfeatures_201711shp/nynta.shp"
coord_system = {'init': 'epsg:4326'}
ntas = gpd.read_file(fname).to_crs(coord_system)
#Keep wanted boroughs and Reset index
ntas = ntas.loc[ntas['BoroName'].isin(['Queens','Brooklyn', 'Manhattan'])]

#Reindex
ntas=ntas.reset_index(drop=True)
#Rename NTACode to GeoID to merge with economic data(see below)
ntas.rename(columns={'NTACode': 'GeoID'}, inplace=True)
ntas.rename(columns={'NTAName': 'Neighborhood'}, inplace = True)
nta = ntas[['Neighborhood', 'GeoID', 'geometry']]

# base = ntas.plot(figsize=(500,20), color='white', edgecolor='black')
# plt.show()

df = pd.read_csv('Median Income Added/April_2016_MDincome.csv', nrows = 21)
start_date_time = pd.to_datetime(df['Date'] + ' ' + df['start_time'])
end_date_time = pd.to_datetime(df['Date'] + ' ' + df['stop_time'])

df['start_date_time'] = start_date_time
df['end_date_time'] = end_date_time
# crs = {'init': 'epsg:4326'}
# gdf = gpd.GeoDataFrame(df, crs=crs)

In [3]:

cc = cartoframes.CartoContext(base_url='https://jgeman.carto.com/',
                              api_key='1f4153a8667cb0f17bfe21bc1d2ccd639304547f')
cc.write(df, 'Start April Median',
         content_guessing=True,
         create_vis = True,
         lnglat=('start_station_longitude', 'start_station_latitude'),
         overwrite=True)

cc.write(df, 'End April Median',
         content_guessing=True,
         lnglat=('end_station_longitude', 'end_station_latitude'),
         overwrite=True)

cc.write(nta, 'NTA',
        content_guessing= True,
        encode_geom = True,
        overwrite = True)

cc.map(layers=[Layer('NTA', color='#C0C0C0'),Layer('Start_April_Median'),
               Layer('End_April_Median')],
       interactive=True)

/Users/stuartgeman/anaconda/lib/python3.6/site-packages/carto/resources.py:90: FutureWarning: This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment
  warnings.warn('This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment', FutureWarning)
/Users/stuartgeman/anaconda/lib/python3.6/site-packages/cartoframes/context.py:267: UserWarning: Table will be named `start_april_median`
  warn('Table will be named `{}`'.format(table_name))


The following columns were changed in the CARTO copy of this dataframe:
start station id -> start_station_id
start station name -> start_station_name
start station latitude -> start_station_latitude
start station longitude -> start_station_longitude
end station id -> end_station_id
end station name -> end_station_name
end station latitude -> end_station_latitude
end station longitude -> end_station_longitude
birth year -> birth_year
Date -> date
Time -> time
TempF -> tempf
WeatherDesc -> weatherdesc
WindSpeedMiles -> windspeedmiles
Day -> day
start point -> start_point
end point -> end_point
start neighborhood -> start_neighborhood
start borough -> start_borough
start median income -> start_median_income
end neighborhood -> end_neighborhood
end borough -> end_borough
end median income -> end_median_income
Table successfully written to CARTO: https://jgeman.carto.com/dataset/start_april_median
The following columns were changed in the CARTO copy of this dataframe:
start station id -> st

/Users/stuartgeman/anaconda/lib/python3.6/site-packages/cartoframes/context.py:267: UserWarning: Table will be named `end_april_median`
  warn('Table will be named `{}`'.format(table_name))


Table successfully written to CARTO: https://jgeman.carto.com/dataset/end_april_median
The following columns were changed in the CARTO copy of this dataframe:
Neighborhood -> neighborhood
GeoID -> geoid


/Users/stuartgeman/anaconda/lib/python3.6/site-packages/cartoframes/context.py:267: UserWarning: Table will be named `nta`
  warn('Table will be named `{}`'.format(table_name))


Table successfully written to CARTO: https://jgeman.carto.com/dataset/nta


In [4]:
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))

In [17]:
from IPython.core.display import HTML
HTML('<a href="file:///Users/stuartgeman/Desktop/poop.html>link</a>')

[link text](./poop.html)

In [19]:
from cartoframes import QueryLayer
cc.map(layers=QueryLayer('''
    SELECT *, 'start' as station_type FROM Start_April_Median
    UNION ALL
    SELECT *, 'end' as station_type FROM End_April_Median
''', type = 'torque',
     color='station_type'))

TypeError: __init__() got an unexpected keyword argument 'type'

In [14]:
# Neighborhood Tabulation Area defined neighborhoods (Census tract Stuff)
fname = "nycgiszoningfeatures_201711shp/nynta.shp"
coord_system = {'init': 'epsg:4326'}
ntas = gpd.read_file(fname).to_crs(coord_system)
#Keep wanted boroughs and Reset index
ntas = ntas.loc[ntas['BoroName'].isin(['Queens','Brooklyn', 'Manhattan'])]

#Reindex
ntas=ntas.reset_index(drop=True)
#Rename NTACode to GeoID to merge with economic data(see below)
ntas.rename(columns={'NTACode': 'GeoID'}, inplace=True)
ntas.rename(columns={'NTAName': 'Neighborhood'}, inplace = True)
nta = ntas[['Neighborhood', 'GeoID', 'geometry']]

# base = ntas.plot(figsize=(500,20), color='white', edgecolor='black')
# plt.show()

#A dataframe with young people(born after 1988) going to to work in the morning 
df = pd.read_csv('Income and Bike Id/april_med_bike_id.csv')
df = df[(df['time'] >= 600) & (df['time'] <=900) & (df['birth year'] >= 1988.0) & (df['date']== '2016-04-01')]

#Create datetime columns so you can animate your maps based on Carto's required date/time format
start_date_time = pd.to_datetime(df['date'] + ' ' + df['start_time'])
end_date_time = pd.to_datetime(df['date'] + ' ' + df['stop_time'])

df['start_date_time'] = start_date_time
df['end_date_time'] = end_date_time
# crs = {'init': 'epsg:4326'}
# gdf = gpd.GeoDataFrame(df, crs=crs)


cc = cartoframes.CartoContext(base_url='https://jgeman.carto.com/',
                              api_key='1f4153a8667cb0f17bfe21bc1d2ccd639304547f')

cc.write(df, '88+ Morning Commute April',
         content_guessing=True,
         create_vis = True,
         lnglat=('start_station_longitude', 'start_station_latitude'),
         overwrite=True)

/Users/stuartgeman/anaconda/lib/python3.6/site-packages/carto/resources.py:90: FutureWarning: This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment
  warnings.warn('This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment', FutureWarning)
/Users/stuartgeman/anaconda/lib/python3.6/site-packages/cartoframes/context.py:267: UserWarning: Table will be named `_88_morning_commute_april`
  warn('Table will be named `{}`'.format(table_name))


The following columns were changed in the CARTO copy of this dataframe:
start station id -> start_station_id
start station name -> start_station_name
start station latitude -> start_station_latitude
start station longitude -> start_station_longitude
end station id -> end_station_id
end station name -> end_station_name
end station latitude -> end_station_latitude
end station longitude -> end_station_longitude
birth year -> birth_year
start point -> start_point
end point -> end_point
start neighborhood -> start_neighborhood
start borough -> start_borough
start median income -> start_median_income
end neighborhood -> end_neighborhood
end borough -> end_borough
end median income -> end_median_income
Table successfully written to CARTO: https://jgeman.carto.com/dataset/_88_morning_commute_april
